In [1]:
import shelve
from time import sleep
import vk_api
from vk_api import VkUpload
from vk_api.longpoll import VkLongPoll, VkEventType
import requests
from vk_api.execute import VkFunction
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib
import json
import datetime
import numpy as np
import shelve


import settings

In [2]:
def isCommunityAlive(comm_id, vk):
    try:
        info = vk.wall.get(owner_id=-comm_id, owners_only=1, count=2)
        try:
            date1 = datetime.datetime.fromtimestamp(info.get('items')[0].get('date'))
        except:
            return (comm_id,  "Dead") #No posts
                                                    
        try: #the only one post
            date2 = datetime.datetime.fromtimestamp(info.get('items')[1].get('date'))
            last_post_date = max(date1, date2)
        except:
            last_post_date = date1
                             
        last_count = datetime.datetime.now() - last_post_date
        if (last_count.days < 30):
            return (comm_id, "Alive")
        else:
            return (comm_id, "Dead")
    except vk_api.exceptions.ApiError as e:
        return (comm_id, e.__str__())

In [12]:
class DA_downloader:
    def __init__(self, to_download_file='to_download.txt',
                 already_downloaded_file='already_downloaded.txt'):
        
        self.to_download_file = to_download_file
        self.already_downloaded_file = already_downloaded_file
        
        with open(self.to_download_file) as f:
            self.download_list = f.readlines()
            self.download_list = [int(x.strip()) for x in self.download_list]
        
        with open(self.already_downloaded_file) as f:
            self.already_downloaded = f.readlines()
            self.already_downloaded = [int(x.strip()) for x in self.already_downloaded]

        
    def get_by_token(self, token):
        vk_session = vk_api.VkApi(token=token)
        try:
            vk_session.auth(reauth=True, token_only=True)
        except vk_api.AuthError as error_msg:
            print(error_msg)
    
        vk = vk_session.get_api()

        ans = []
        for group_id in self.download_list:
            if group_id not in self.already_downloaded:
                ans.append(isCommunityAlive(group_id, vk))
                if ans[-1][1] == '[29] Rate limit reached':
                    print('[29] Rate limit reached')
                    break

        if len(ans) > 1:
            dl_append = [id for id, status in ans if status != '[29] Rate limit reached']
            self.already_downloaded += dl_append
        
            with open(self.already_downloaded_file, 'a') as f:
                for item in dl_append:
                    f.write("%s\n" % item)

        
            File = shelve.open(f"data_batch_from{ans[0][0]}_to{ans[-2][0]}")
            self.batch_count += 1
            File['info'] = ans[:-1]
            File.close()
        
            print(f'{len(dl_append)} analyzed')
        else:
            print(f'Bad responce')
        return ans
    
    def get_by_login(self, login=settings.MY_USER_LOGIN, password=settings.MY_USER_PASSWORD):
        vk_session = vk_api.VkApi(login=login, password=password)
        try:
            vk_session.auth(reauth=True, token_only=True)
        except vk_api.AuthError as error_msg:
            print(error_msg)
    
        vk = vk_session.get_api()

        ans = []
        for group_id in self.download_list:
            if group_id not in self.already_downloaded:
                ans.append(isCommunityAlive(group_id, vk))
                if ans[-1][1] == '[29] Rate limit reached':
                    print('[29] Rate limit reached')
                    break

        if len(ans) > 1:
            dl_append = [id for id, status in ans if status != '[29] Rate limit reached']
            self.already_downloaded += dl_append
        
            with open(self.already_downloaded_file, 'a') as f:
                for item in dl_append:
                    f.write("%s\n" % item)

        
            File = shelve.open(f"data_batch_from{ans[0][0]}_to{ans[-2][0]}")
            File['info'] = ans[:-1]
            File.close()
        
            print(f'{len(dl_append)} analyzed')
        else:
            print(f'Bad responce')
        return ans
    
    def is_finished(self):
        return len(self.already_downloaded) == len(self.download_list)

In [13]:
%%time

DA_dld = DA_downloader()
DA_dld.get_by_login()
DA_dld.get_by_login()

[29] Rate limit reached
Bad responce
[29] Rate limit reached
Bad responce
CPU times: user 13.8 s, sys: 17.4 ms, total: 13.8 s
Wall time: 14.6 s


In [7]:
%%time

token_list = ['fda37960fda37960fda3796002fdc4564affda3fda37960a6592150811214b3b116ce40']

DA_dld = DA_downloader()

for token in token_list:
    DA_dld.get_by_token(token)

Login is required to auth
[29] Rate limit reached
5000 analyzed
CPU times: user 34.4 s, sys: 965 ms, total: 35.3 s
Wall time: 35min 32s


In [14]:
with open('already_downloaded.txt') as f:
    already_downloaded = f.readlines()
    already_downloaded = [int(x.strip()) for x in already_downloaded]
    
already_d = already_downloaded[:-5000]

with open('already_downloaded.txt', 'w') as f:
    for item in already_d:
        f.write("%s\n" % item)

In [15]:
len(already_d)

39369